In [21]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Wed Jun 16 21:27:50 2021

@author: xiaoxu
"""


import sympy as smp
from sympy import Number, Add, Mul, Pow, Basic, latex, Symbol, I
from sympy.physics.secondquant import  Dagger
from copy import deepcopy
from printable import Basic as Printable,  B, Bd
from terms import Term, Terms, Annilator, Creator, Exp, t
import pickle

class Kamiltonian(Terms):

    Ks = {}

    @classmethod
    def get(cls, n, k):
        """
        get Kamiltonian K(n)_[k] according to the recursive formula.
        """
        key = str([n,k])
        # print(key)

        if key in cls.Ks.keys():
            return cls.Ks[key]

        if n == 0 and k == 0:#base case
            raise Exception("Base case of recursion not specified")

        elif n == 0 and k == 1:
            cls.Ks[key] = S(1).dot().simplify()

        elif n != 0 and k == 1:
            term1 = S(n+1).dot()
            term2 = S(n).lie_der(K(0,0))

            cls.Ks[key] = (term1 + term2).simplify()
            #this can be alternatively expressed as static of term2 - rotating
            #parts of other K(n,k'!=k)

        elif k > 1 and k <= n+1:
            # print("general")
            terms = Terms()
            for m in range(n):
                # print(m,k-1)
                terms += S(n-m).lie_der(K(m,k-1))/smp.S(k)
            cls.Ks[key] = terms.simplify()
        else:
            # print("zero case")
            return Terms()
        # cls.Ks[key].simplify()
        return cls.Ks[key]

    @classmethod
    def set_H(cls, H):
        cls.Ks[str([0,0])] = H.simplify()

    @classmethod
    def dump(cls, fdir):
        with open(fdir,'wb') as f:
            pickle.dump(cls.Ks,f)

    @classmethod
    def load(cls, fdir):
        with open(fdir,'rb') as f:
            cls.Ks = pickle.load(f)


class Generator(Terms):
    Ss = {}

    @classmethod
    def get(cls, n):
        """
        get generator S(n) according to the recursive formula.
        """
        if n in cls.Ss.keys(): return cls.Ss[n]

        # print("S"+str(n))
        terms = Terms()
        if n == 0: return Terms()
        if n == 1:
            terms += smp.S(-1)*K(0,0)
        if n > 1:
            terms += smp.S(-1)*S(n-1).lie_der(K(0,0))
        for k in range(2, n+1):
            # print(k, n-1)
            terms += smp.S(-1)*K(n-1,k)
        terms = terms.rot().integrate()

        cls.Ss[n] = terms.simplify()
        # print("S"+str(n)+"done")
        return cls.Ss[n]

    @classmethod
    def dump(cls, fdir):
        with open(fdir,'wb') as f:
            pickle.dump(cls.Ss,f)

    @classmethod
    def load(cls, fdir):
        with open(fdir,'rb') as f:
            cls.Ss = pickle.load(f)


def K(n,k = -1):
    if k!= -1: return Kamiltonian.get(n,k)
    Kn = Terms()
    for ki in range(n+2):
        Kn += Kamiltonian.get(n,ki)
    return Kn

def S(n):
    return Generator.get(n)

# Term.set_lambda(smp.S(1))

hbar = Symbol("hbar", real=True)

omega0 = Symbol('\omega_a')
omega1 = Symbol('\omega_b')
g3 = Symbol('g_3')
g4 = Symbol('g_4')
g5 = Symbol('g_5')
g6 = Symbol('g_6')
g7 = Symbol('g_7')
g8 = Symbol('g_8')
delta = Symbol('\delta')



#b = Annilator('a')*Annilator('b')
#br = b*Exp(-1*I*omega1*t)
#bd = Creator('a')
#bdr = bd*Exp(1*I*omega1*t)

a = Annilator('a')
ar = a*Exp(-1*I*omega0*t)
ad = Creator('a')
adr = ad*Exp(1*I*omega0*t)

#pi = Symbol('\Pi ', complex=True)
#pir = pi*Exp(-1*I*(2*omega0)*t)
#pis = pi.conjugate()
#pisr = pis*Exp(1*I*(2*omega0)*t)
H =  g4*((ar+adr)**4) + g3*((ar+adr)**3) +g5*(ar+adr)**5+g6*(ar+adr)**6
#H = g3*(ar+adr)**3+ g4*(ar+adr)**4

Kamiltonian.set_H(H)




In [22]:
(K(0)).simplify()

Terms(Term(Prefactor((6*g_4 + 90*g_6*hbar, 1)), Bd(a)**2*B(a)**2, 1), Term(Prefactor((12*g_4*hbar + 90*g_6*hbar**2, 1)), Bd(a)*B(a), 1), Term(Prefactor((3*g_4*hbar**2 + 15*g_6*hbar**3, 1)), 1, 1), Term(Prefactor((20*g_6, 1)), Bd(a)**3*B(a)**3, 1))

In [23]:
(K(1)+K(0)).simplify()

Terms(Term(Prefactor((-11*g_3**2*hbar**2 - 130*g_3*g_5*hbar**3 - 42*g_4**2*hbar**3 - 720*g_4*g_6*hbar**4 - 449*g_5**2*hbar**4 - 3495*g_6**2*hbar**5, \omega_a/hbar), (3*g_4*hbar**2 + 15*g_6*hbar**3, 1)), 1, 1), Term(Prefactor((-660*g_4*g_6 - 630*g_5**2 - 19650*g_6**2*hbar, \omega_a/hbar)), Bd(a)**4*B(a)**4, 1), Term(Prefactor((-280*g_3*g_5 - 68*g_4**2 - 5280*g_4*g_6*hbar - 5040*g_5**2*hbar - 75100*g_6**2*hbar**2, \omega_a/hbar), (20*g_6, 1)), Bd(a)**3*B(a)**3, 1), Term(Prefactor((-30*g_3**2 - 1260*g_3*g_5*hbar - 306*g_4**2*hbar - 11340*g_4*g_6*hbar**2 - 10220*g_5**2*hbar**2 - 102150*g_6**2*hbar**3, \omega_a/hbar), (6*g_4 + 90*g_6*hbar, 1)), Bd(a)**2*B(a)**2, 1), Term(Prefactor((-60*g_3**2*hbar - 1100*g_3*g_5*hbar**2 - 288*g_4**2*hbar**2 - 6840*g_4*g_6*hbar**3 - 5320*g_5**2*hbar**3 - 43650*g_6**2*hbar**4, \omega_a/hbar), (12*g_4*hbar + 90*g_6*hbar**2, 1)), Bd(a)*B(a), 1), Term(Prefactor((-1572*g_6**2, \omega_a/hbar)), Bd(a)**5*B(a)**5, 1))

In [6]:
(K(2)+K(1)+K(0)).select(ad**2*a**2).simplify()

Terms(Term(Prefactor((8100*g_3**2*g_4*hbar + 174540*g_3**2*g_6*hbar**2 + 309480*g_3*g_4*g_5*hbar**2 + 6153480*g_3*g_5*g_6*hbar**3 + 25164*g_4**3*hbar**2 + 1564560*g_4**2*g_6*hbar**3 + 2714040*g_4*g_5**2*hbar**3 + 32148900*g_4*g_6**2*hbar**4 + 52888720*g_5**2*g_6*hbar**4 + 222536700*g_6**3*hbar**5, \omega_a**2), (-30*g_3**2 - 1260*g_3*g_5*hbar - 306*g_4**2*hbar - 11340*g_4*g_6*hbar**2 - 10220*g_5**2*hbar**2 - 102150*g_6**2*hbar**3, \omega_a), (6*g_4 + 90*g_6*hbar, 1)), Bd(a)**2*B(a)**2, 1))

In [7]:
(K(3)+K(2)+K(1)+K(0)).select(ad**2*a**2).simplify()

Terms(Term(Prefactor((-12690*g_3**4*hbar - 1228200*g_3**3*g_5*hbar**2 - 1570836*g_3**2*g_4**2*hbar**2 - 69297120*g_3**2*g_4*g_6*hbar**3 - 38631600*g_3**2*g_5**2*hbar**3 - 743772276*g_3**2*g_6**2*hbar**4 - 61178400*g_3*g_4**2*g_5*hbar**3 - 2623479024*g_3*g_4*g_5*g_6*hbar**4 - 508880960*g_3*g_5**3*hbar**4 - 28135003320*g_3*g_5*g_6**2*hbar**5 - 2566170*g_4**4*hbar**3 - 231766920*g_4**3*g_6*hbar**4 - 575850232*g_4**2*g_5**2*hbar**4 - 7865125920*g_4**2*g_6**2*hbar**5 - 24664631760*g_4*g_5**2*g_6*hbar**5 - 120253555800*g_4*g_6**3*hbar**6 - 2459114280*g_5**4*hbar**5 - 268277860040*g_5**2*g_6**2*hbar**6 - 704235410550*g_6**4*hbar**7, \omega_a**3), (8100*g_3**2*g_4*hbar + 174540*g_3**2*g_6*hbar**2 + 309480*g_3*g_4*g_5*hbar**2 + 6153480*g_3*g_5*g_6*hbar**3 + 25164*g_4**3*hbar**2 + 1564560*g_4**2*g_6*hbar**3 + 2714040*g_4*g_5**2*hbar**3 + 32148900*g_4*g_6**2*hbar**4 + 52888720*g_5**2*g_6*hbar**4 + 222536700*g_6**3*hbar**5, \omega_a**2), (-30*g_3**2 - 1260*g_3*g_5*hbar - 306*g_4**2*hbar - 11340*g_

In [ ]:
(K(4)+K(3)+K(2)+K(1)+K(0)).select(ad**2*a**2).simplify()